___
##### $Name:\,\color{blue}{Christopher\,J.\,Watson\,,Nathan\,Edwards\,,Paul\,Thai}$
##### $School:\,\color{blue}{Marcos\,School\,of\,Engineering,\,University\,of\,San\,Diego}$
##### $Class:\,\color{blue}{AAI\,511-\,Neural\,Networks\,and\,Learning}$
##### $Assignment:\,\color{blue}{MSAAI\,Final\,Project}$
##### $Date:\,\color{blue}{8/15/2023}$
___

### Libraries
____

In [1]:
# Libraries 

# File and Data Handling
import os

# Data Visualization and Numerical Computing
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# MIDI Processing
import mido

# Machine Learning and Deep Learning
from torch.utils.data import Dataset, DataLoader
import torch
from torch import nn, optim

# Machine learning Metrics and Evaluations
from sklearn.metrics import accuracy_score, precision_score, f1_score, confusion_matrix

### Classes
___

In [2]:
class Dataset(Dataset):
    
    def __init__(self, sequences, labels):
        self.sequences = sequences
        self.labels = labels

    def enum(self,y):
        composers = ["bach", "bartok", "byrd", "chopin", "handel", "hummel", "mendelssohn","mozart", "schumann"]
        #print(y)
        for i in range(len(composers)):
            if composers[i] == y:
                slot = i
        out = [0] * 9
        out[slot] = 1
        
        return out

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        sequence = self.sequences[idx]
        label = self.labels[idx]
        label = self.enum(label)
           
        
        sequence = torch.tensor(sequence)
        label = torch.tensor(label)
        sequence = sequence[None, :, :] 
        return sequence, label

### Helper Functions
___

In [3]:
def get_children(a_dir):
    dirs = []
    files = []
    for name in os.listdir(a_dir):
        if os.path.isdir(os.path.join(a_dir, name)):
            dirs.append(name)
        else:
            files.append(name)
    return [dirs,files]

def create_files_table(top_level, out_file):
    temp_comps = []
    temp_songs = []
    temp_paths = []
    
    composer_names, songs = get_children(top_level)

    for composer in composer_names:
        temp_path = top_level + '/' + composer
        temp, songs = get_children(temp_path)
        for song in songs:
            if song != '.DS_Store':
                temp_comps.append(composer)
                temp_paths.append(temp_path + '/' + song)
                temp_songs.append(song.split(".")[0])

    temp_dict = {'Composers': temp_comps, 'Songs': temp_songs, 'Paths': temp_paths}

    table = pd.DataFrame.from_dict(temp_dict)

    table.to_csv('./' + out_file + '.csv',index=False)
    
    return table


# Function to extract the notes played in a MIDI file with timestamps
def extract_notes_with_meta(midi_filepath):
    notes = {}
    midi = mido.MidiFile(midi_filepath)
    max_time = 0
    time_counter = 0
    for track in midi.tracks:
        max_time = max(max_time,time_counter)
        time_counter = 0
        for msg in track:
            time_counter += msg.time
            
            if msg.type == 'note_on':
                if msg.velocity != 0:  # Ensure it's a Note On event
                    notes[msg.note] = notes.get(msg.note, []) + [(msg.velocity, time_counter, 1)]  # 1 represents Note On
            elif msg.type == 'note_off':
                notes[msg.note] = notes.get(msg.note, []) + [(msg.velocity,time_counter, 0)]  # 0 represents Note Off
                
    return notes, max_time


def create_single_sequences(notes, start, tick_count, seq_count): 
    VEL = 0
    TM = 1
    ON = 2
    
    temp_keys = notes.keys()

    seq =  [[0] * 128]* seq_count
    seq = np.array(seq)

    for x in temp_keys:
        temp_note = np.array(notes[x])
        time_store = 0
        for i in range(start, seq_count+start):
            temp_vel = 0
            for t in range(time_store,temp_note[:,TM].size):
                if (temp_note[t,TM]>tick_count*i):
                    break
                else:
                    if temp_note[t,ON] == 1:
                        if temp_note[t,VEL] > temp_vel:
                            temp_vel = temp_note[t,VEL]
                time_store = t
            seq[i-start,x] = temp_vel
    return seq

def sequence_songs(df_songs, tick_count, seq_count, jiggle_on=False):
    labels = []
    sequences = []
    
    
    shake_amount = 0
    
    if jiggle_on:
        shake_amount = [0, int(seq_count/2), int(seq_count/4)]
    
    for j in shake_amount:
        for song in df_songs.iterrows():
            song = song[1]
            notes, max_time = extract_notes_with_meta(song['Paths'])
            for i in range(int((max_time-j)/(seq_count*tick_count))):
                sequences.append(create_single_sequences(notes, i*seq_count+j, tick_count, seq_count))
                labels.append(song['Composers'])
                
    return labels, sequences

### Preprocessing
___

In [4]:
devpath = './Composer_Dataset/NN_midi_files_extended/dev/'
testpath = './Composer_Dataset/NN_midi_files_extended/test/'
trainpath = './Composer_Dataset/NN_midi_files_extended/train/'

dev_table = create_files_table(devpath, 'dev_table')
test_table = create_files_table(testpath, 'test_table')
train_table = create_files_table(trainpath, 'train_table')

In [5]:
midi_file_path = './Composer_Dataset/NN_midi_files_extended/dev/bach/bach344.mid'

In [8]:
labels, sequences = sequence_songs(train_table, 200, 128,jiggle_on=True)

In [9]:
labels_test, sequences_test = sequence_songs(test_table, 200, 128,jiggle_on=True)

In [10]:
# Creating the dataset for the dataloader
dataset = Dataset(sequences, labels)
dataset_test = Dataset(sequences_test, labels_test)

# Creating data loader
batch_size = 1

data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset_test, batch_size=batch_size, shuffle=False)

In [ ]:
dataset[0][0].shape

In [ ]:
len(test_loader)

### Model Creation
___

In [14]:
# Model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 4, kernel_size=2, padding=1)
        self.conv2 = nn.Conv2d(4, 8, kernel_size=2, padding=1)
        self.conv3 = nn.Conv2d(8, 16, kernel_size=2, padding=1)
        self.conv4 = nn.Conv2d(16, 32, kernel_size=2, padding=1)


        self.pool = nn.MaxPool2d(kernel_size=2)

        self.LSTM = nn.LSTM(8, 50, 3, batch_first=True)

        self.fc1 = nn.Linear(400, 200)
        self.fc2 = nn.Linear(200, 100)  
        self.fc3 = nn.Linear(100, 9)  

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = self.pool(nn.functional.relu(self.conv3(x)))
        x = self.pool(nn.functional.relu(self.conv4(x)))

        x =  x[:,0,:,:]
        h0 = torch.zeros(3, x.size(0), 50)
        c0 = torch.zeros(3, x.size(0), 50)        
        x, _ = self.LSTM(x, (h0, c0)) 

        x = torch.flatten(x, 1)

        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = self.fc3(x)
        # return nn.functional.sigmoid(x)
        return nn.functional.log_softmax(x, dim=1)
# class CNN(nn.Module):
#     def __init__(self):
#         super(CNN, self).__init__()
#         self.conv1 = nn.Conv2d(16, 8, kernel_size=3, padding=1)
#         self.conv2 = nn.Conv2d(8, 1, kernel_size=3, padding=1)
#         self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
#         self.LSTM= nn.LSTM(32, 10, 1, batch_first=True)
                           
#         self.fc1 = nn.Linear(2500, 128) 
#         self.fc3 = nn.Linear(128, 9)  

#     def forward(self, x):
#         x = self.pool(nn.functional.relu(self.conv1(x)))
#         x = self.pool(nn.functional.relu(self.conv2(x)))
#         #print(x.shape)         
#         x =  x[:,0,:,:]
#         h0 = torch.zeros(1, x.size(0), 10)
#         c0 = torch.zeros(1, x.size(0), 10)        
#         x, _ = self.LSTM(x, (h0, c0))               
                                       
        
        
#         x = torch.flatten(x, 1)
#         x = nn.functional.relu(self.fc1(x))
#         x = self.fc3(x)
#         return nn.functional.sigmoid(x)


In [15]:
# Create model object
model = CNN() 

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adagrad(model.parameters(),lr=0.001)

### Training
___

In [16]:
# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    model.train() 
    running_loss = 0.0
    for inputs, labels in data_loader:
        # Zero the parameter gradients
        optimizer.zero_grad()

        inputs = inputs.to(torch.float)
        labels = labels.to(torch.float)

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Print the average loss for the epoch
    epoch_loss = running_loss / len(data_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}] Loss: {epoch_loss:.4f}")

print("Training complete!")

Epoch [1/5] Loss: 5.4216
Epoch [2/5] Loss: 5.4194
Epoch [3/5] Loss: 5.4183
Epoch [4/5] Loss: 5.4176
Epoch [5/5] Loss: 5.4170
Training complete!


### Evaulation and Results
___

In [17]:
# Evaluating the model
model.eval()

prediction_list = list()
labels_list = list()

real = []
pred = []
counter = 0
# Disable gradient computation to save memory
with torch.no_grad():
    for inputs, labels in test_loader:
        
        # Forward pass
        outputs = model(inputs.float())
        _, predicted = torch.max(outputs.data, 1)
        
        print(outputs)
        
        outputs = outputs.tolist()
        labels = labels.tolist()
        
        pred_labal = outputs[0].index(max(outputs[0]))
        

        print(pred_labal)
        
        print(labels)
        
        real_label = labels[0].index(max(labels[0]))
        
        real.append(real_label)
        pred.append(pred_labal)


        


tensor([[-1.9646, -2.2278, -2.2344, -2.2326, -2.2218, -2.2260, -2.2321, -2.2371,
         -2.2323]])
0
[[1, 0, 0, 0, 0, 0, 0, 0, 0]]
tensor([[-1.9563, -2.2272, -2.2327, -2.2304, -2.2187, -2.2390, -2.2279, -2.2428,
         -2.2364]])
0
[[1, 0, 0, 0, 0, 0, 0, 0, 0]]
tensor([[-2.0232, -2.2180, -2.2353, -2.2359, -2.1963, -2.1986, -2.2427, -2.2169,
         -2.2277]])
0
[[1, 0, 0, 0, 0, 0, 0, 0, 0]]
tensor([[-1.9311, -2.2284, -2.2320, -2.2328, -2.2341, -2.2500, -2.2281, -2.2419,
         -2.2417]])
0
[[1, 0, 0, 0, 0, 0, 0, 0, 0]]
tensor([[-2.0304, -2.2007, -2.2324, -2.2332, -2.1878, -2.2124, -2.2418, -2.2214,
         -2.2330]])
0
[[1, 0, 0, 0, 0, 0, 0, 0, 0]]
tensor([[-1.9249, -2.2334, -2.2314, -2.2353, -2.2388, -2.2512, -2.2248, -2.2434,
         -2.2390]])
0
[[1, 0, 0, 0, 0, 0, 0, 0, 0]]
tensor([[-1.9263, -2.2293, -2.2302, -2.2323, -2.2338, -2.2596, -2.2218, -2.2469,
         -2.2416]])
0
[[1, 0, 0, 0, 0, 0, 0, 0, 0]]
tensor([[-2.0202, -2.2139, -2.2347, -2.2323, -2.1878, -2.2043, -2.237

In [18]:
print("The model had a accuracy_score socre of : ", round(accuracy_score(pred, real),4))

The model had a accuracy_score socre of :  0.4405
